In [2]:
!pip install kafka-python

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from kafka import KafkaConsumer
import json
 
def kafka_consumer_example():
    consumer = KafkaConsumer(
        'cf_CP',
        bootstrap_servers = ['master:9092'],
        auto_offset_reset = 'earliest',
        enable_auto_commit = True
    )
    bytes_to_string=lambda byte_data,encoding='utf-8':byte_data.decode(encoding)
    print("Waiting for messages...")
    for message in consumer:
        val=bytes_to_string(message.value)
        print(f"Recieved message length: {len(val)}")
        print(f"Recieved message : {message.value}")
    consumer.close()
if __name__ == "__main__":
    kafka_consumer_example()
    
#kafka-console-producer.sh --topic coforge --broker-list master:9092
#run above command in terminal to write message

#kafka-console-consumer.sh --bootstrap-server master:9092 --topic coforge --from-beginning
#above command will show all the messages in terminal

#to delete the topic
# kafka-topics.sh --bootstrap-server master:9092 --delete --topic

In [ ]:
from kafka import KafkaProducer
import json

def kafka_producer_example():
    # Initialize Kafka producer
    producer = KafkaProducer(
        bootstrap_servers=['master:9092'],  # Replace with your Kafka broker address
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )
    
    # Send a message
    for i in range(10):
        topic_name = 'sub_1'
        message = {'key': i}  # Replace with your actual message
        producer.send(topic_name, message)
        producer.flush()  # Ensure all messages are sent before exiting

        print(f"Message sent to {topic_name}: {message}")
    producer.close()

if __name__ == "__main__":
    kafka_producer_example()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
# Initialize SparkSession with Kafka dependencies
spark = SparkSession.builder \
    .appName("KafkaStructuredStreamingExample") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2") \
    .config("spark.yarn.jars", "hdfs:///opt/spark/*.jar") \
    .getOrCreate()
# Kafka configuration details
kafka_bootstrap_servers = "master:9092"  # Replace with your Kafka broker address
kafka_topic = "gfg"  # Replace with your Kafka topic
# Read data from Kafka
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets","earliest") \
    .load()
# Extract the 'value' column from the Kafka message and cast it to a string
kafka_df = df.selectExpr("CAST(value AS STRING)")
# Write the output to the console (this works in notebook output)
query = kafka_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()
# Await termination (in a notebook, you might want to limit this or use a specific timeout)
# In a Jupyter notebook, we use a short timeout to avoid blocking the notebook for too long
query.awaitTermination(60)
has context menu